In [ ]:
from common_imports import *

In [ ]:
run_detector1_pipeline = True

In [ ]:
############ STAGE 1 DETECTOR PIPELINE

# Define a function to call the detector1 pipeline with desired parameters.
# Individual steps not enumerated. Options can be set as the commented overrides.

def rundet1(filename, outdir):
    """
    Create a Detector1Pipeline object and set all the desired stage 1
    pipeline processing arguments.
    
    Arguments:
      * filename  :  _uncal.fits exposure file name
      * outdir    :  Directory for the stage 1 output files
    
    Returns:
      * None
      
    Outputs:
      * Writes a _rate.fits and _rateints.fits file to `outdir`.
    """
    
    print('Running Detector1Pipeline on {0:s}...'.format(filename))
    
    det1 = Detector1Pipeline() # Instantiate the pipeline
    det1.output_dir = outdir   # Specify where the output should go
    
    # Overrides for whether or not certain steps should be skipped
    #det1.dq_init.skip = False
    #det1.saturation.skip = False
    #det1.firstframe.skip = False
    #det1.lastframe.skip = False
    #det1.reset.skip = False
    #det1.linearity.skip = False
    #det1.rscd.skip = False
    #det1.dark_current.skip = False
    #det1.refpix.skip = False
    #det1.jump.skip = False
    #det1.ramp_fit.skip = False
    #det1.gain_scale.skip = False
    
    # Trun on multiprocessing for jump and ramp fitting steps
    det1.jump.maximum_cores = 'half'
    det1.ramp_fit.maximum_cores = 'half'
    
    # Bad pixel mask overrides
    #det1.dq_init.override_mask = 'myfile.fits'

    # Saturation overrides
    #et1.saturation.override_saturation = 'myfile.fits'
    
    # Reset overrides
    #det1.reset.override_reset = 'myfile.fits'
        
    # Linearity overrides
    #det1.linearity.override_linearity = 'myfile.fits'

    # RSCD overrides
    #det1.rscd.override_rscd = 'myfile.fits'
        
    # DARK overrides
    #det1.dark_current.override_dark = 'myfile.fits'
        
    # GAIN overrides
    #det1.jump.override_gain = 'myfile.fits'
    #det1.ramp_fit.override_gain = 'myfile.fits'
                
    # READNOISE overrides
    #det1.jump.override_readnoise = 'myfile.fits'
    #det1.ramp_fit.override_readnoise = 'myfile.fits'
    
    # JUMP overrides. 
    # Currently pipeline is not flagging lower-level jumps
    # like we might want it to, so lower thresholds for more
    # aggressive flagging.
    #det1.jump.save_results = True
    det1.jump.rejection_threshold = 3.5         # default 4.0sigma
    det1.jump.min_jump_to_flag_neighbors = 8.0  # default 10sigma
    
    # Additional JUMP overrides related to CR shower flagging. See
    # JWST pipeline documentation page for details of parameters.
    # https://jwst-pipeline.readthedocs.io/en/stable/jwst/jump/index.html
    det1.jump.expand_large_events   = True   # Turn on shower flagging
    det1.jump.use_ellipses          = True   # True for MIRI; approximate showers as 
    # elliptical, obsolete in newest pipeline
    det1.jump.min_jump_area         = 8     # Min # of contiguous pixels to trigger expanded flagging
    det1.jump.sat_required_snowball = True  # Require pixels to be saturated to flag
    det1.jump.expand_factor         = 3.0    # expands showers beyond ID'd jumps; 
    # default 2.0
    
    det1.jump.after_jump_flag_dn1   = 10     # These 4 related to how long after a jump is identified
    det1.jump.after_jump_flag_time1 = 20     #  we should keep flagging the following integrations
    det1.jump.after_jump_flag_dn2   = 1000
    det1.jump.after_jump_flag_time2 = 3000
    
    ### TEMPLATES settings
    # # JUMP overrides. 
    # # Currently pipeline is not flagging lower-level jumps
    # # like we might want it to, so lower thresholds for more
    # # aggressive flagging.
    # #det1.jump.save_results = True
    # det1.jump.rejection_threshold = 3.5         # default 4.0sigma
    # det1.jump.min_jump_to_flag_neighbors = 8.0  # default 10sigma
    # 
    # # Additional JUMP overrides related to CR shower flagging. See
    # # JWST pipeline documentation page for details of parameters.
    # # https://jwst-pipeline.readthedocs.io/en/stable/jwst/jump/index.html
    # det1.jump.expand_large_events   = True   # Turn on shower flagging
    # det1.jump.use_ellipses          = True   # True for MIRI; approximate showers as elliptical
    # det1.jump.min_jump_area         = 8     # Min # of contiguous pixels to trigger expanded flagging
    # # The saturated core allows the search for smaller events without false positives.
    # det1.jump.sat_required_snowball = False  # Do not require pixels to be saturated to flag
    # det1.jump.expand_factor         = 3.0    # expands showers beyond ID'd jumps; default 2.0
    # 
    # det1.jump.after_jump_flag_dn1   = 10     # These 4 related to how long after a jump is identified
    # det1.jump.after_jump_flag_time1 = 20     #  we should keep flagging the following integrations
    # det1.jump.after_jump_flag_dn2   = 1000
    # det1.jump.after_jump_flag_time2 = 3000
    ### TEMPLATES settings end
    
    # Ramp_fit overrides.
    #det1.ramp_fit.save_opt = True
    #det1.ramp_fit.save_results = True
        
    det1.save_results = True # Save the final resulting _rate.fits files
    
    det1(filename)           # Run the pipeline on an input list of files

In [ ]:
import glob
from tqdm.notebook import tqdm
# Run the Detector1 pipeline on each uncal exposure.

# grab a list of all uncal files from the folder defined above
files_uncal = sorted(glob.glob(uncal+'*nrs1_uncal.fits'))

if run_detector1_pipeline:
    for exposure in tqdm(files_uncal): 
        rundet1(filename=exposure, outdir=stage1)